# Trabalho Prático 3 - Banco de Dados

Karen Leão - 22250541


Ricardo Eliel - 22250556

### Tarefa 1 -  Instalação do PostgreSQL

In [2]:
import getpass
import os

In [ ]:
password = getpass.getpass()
command = "sudo -S apt install postgresql" 
os.system('echo %s | %s' % (password, command))


[sudo] password for hapuque: 



Lendo listas de pacotes...
Construindo árvore de dependências...
Lendo informação de estado...
postgresql is already the newest version (17+264.pgdg22.04+1).
Os seguintes pacotes foram instalados automaticamente e já não são necessários:
  libllvm14 linux-headers-5.15.0-118 linux-headers-5.15.0-118-generic
  linux-headers-5.15.0-119 linux-headers-5.15.0-119-generic
  linux-headers-5.15.0-121 linux-headers-5.15.0-121-generic
  linux-headers-5.15.0-122 linux-headers-5.15.0-122-generic
  linux-image-5.15.0-118-generic linux-image-5.15.0-119-generic
  linux-image-5.15.0-121-generic linux-image-5.15.0-122-generic
  linux-modules-5.15.0-118-generic linux-modules-5.15.0-119-generic
  linux-modules-5.15.0-121-generic linux-modules-5.15.0-122-generic
  linux-modules-extra-5.15.0-118-generic
  linux-modules-extra-5.15.0-119-generic
  linux-modules-extra-5.15.0-121-generic
  linux-modules-extra-5.15.0-122-generic python3-attr python3-docker
  python3-dockerpty python3-docopt python3-dotenv python

0

### Tarefa 2 – Geração de um BD de testes

In [4]:
!git clone https://github.com/foliveirafilho/tpch-pgsql.git

Cloning into 'tpch-pgsql'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 164 (delta 0), reused 0 (delta 0), pack-reused 160 (from 1)
Receiving objects: 100% (164/164), 774.38 KiB | 341.00 KiB/s, done.
Resolving deltas: 100% (69/69), done.


In [9]:
!wget -q https://github.com/electrum/tpch-dbgen/archive/32f1c1b92d1664dba542e927d23d86ffa57aa253.zip -O tpch-dbgen.zip
!unzip -q tpch-dbgen.zip && mv tpch-dbgen-32f1c1b92d1664dba542e927d23d86ffa57aa253 tpch-dbgen && rm tpch-dbgen.zip

In [3]:
password = getpass.getpass()
command = "sudo -S apt-get install -y postgresql postgresql-contrib" 
os.system('echo %s | %s' % (password, command))


[sudo] password for hapuque: 

Lendo listas de pacotes...
Construindo árvore de dependências...
Lendo informação de estado...
postgresql is already the newest version (17+264.pgdg22.04+1).
postgresql-contrib is already the newest version (17+264.pgdg22.04+1).
Os seguintes pacotes foram instalados automaticamente e já não são necessários:
  libllvm14 linux-headers-5.15.0-118 linux-headers-5.15.0-118-generic
  linux-headers-5.15.0-119 linux-headers-5.15.0-119-generic
  linux-headers-5.15.0-121 linux-headers-5.15.0-121-generic
  linux-headers-5.15.0-122 linux-headers-5.15.0-122-generic
  linux-image-5.15.0-118-generic linux-image-5.15.0-119-generic
  linux-image-5.15.0-121-generic linux-image-5.15.0-122-generic
  linux-modules-5.15.0-118-generic linux-modules-5.15.0-119-generic
  linux-modules-5.15.0-121-generic linux-modules-5.15.0-122-generic
  linux-modules-extra-5.15.0-118-generic
  linux-modules-extra-5.15.0-119-generic
  linux-modules-extra-5.15.0-121-generic
  linux-modules-extra-5.15.0-122-generic python3-attr 

0

In [4]:
password = getpass.getpass()
command = "sudo -S -u postgres createuser tpch" 
os.system('echo %s | %s' % (password, command))


[sudo] password for hapuque: could not change directory to "/home/hapuque/Documentos/ufam/bd/tp3": Permission denied
createuser: error: creation of new role failed: ERROR:  role "tpch" already exists


256

##### alterando senha do usuario

In [9]:
!pip install psycopg2-binary


Defaulting to user installation because normal site-packages is not writeable


In [11]:
password = getpass.getpass()
command = "sudo -S -u postgres psql" 
os.system('echo %s | %s' % (password, command))



[sudo] password for hapuque: 

0

In [20]:
from sqlalchemy import create_engine, text

# Configurações de conexão
database = "tpchdb"            # Nome do banco de dados
superuser = "postgres"             # Nome do superusuário
superuser_password = "admin1234"  # Senha do superusuário
host = "localhost"                 # Host do banco de dados (geralmente "localhost")
port = "5432"                      # Porta do PostgreSQL (por padrão é 5432)

# Cria a engine de conexão
engine = create_engine(f"postgresql+psycopg2://{superuser}:{superuser_password}@{host}:{port}/{database}")

# Nome do usuário alvo e a nova senha
usuario_alvo = "tpch"
nova_senha = "admin12345"

# Tenta alterar a senha
try:
    with engine.connect() as connection:
        connection.execute(text(f"ALTER USER {usuario_alvo} PASSWORD :senha"), {"senha": nova_senha})
    print(f"Senha do usuário '{usuario_alvo}' alterada com sucesso.")

except Exception as e:
    print(f"Erro ao tentar alterar a senha: {e}")


Senha do usuário 'tpch' alterada com sucesso.


In [27]:
from sqlalchemy import create_engine, text

# Configurações de conexão com o superusuário
user = "tpch"               # Nome do superusuário
password = "admin12345"  # Senha do superusuário
host = "localhost"                  # Host do banco de dados
port = "5432"                       # Porta padrão do PostgreSQL

# Cria a engine de conexão
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/postgres")

# Consulta para listar os bancos de dados
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT datname FROM pg_database WHERE datistemplate = false;"))
        databases = result.fetchall()
        
    print("Bancos de dados disponíveis:")
    for db in databases:
        print(db[0])

except Exception as e:
    print(f"Erro ao tentar listar os bancos de dados: {e}")


Erro ao tentar listar os bancos de dados: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "tpch"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "tpch"

(Background on this error at: https://sqlalche.me/e/20/e3q8)


In [ ]:
from sqlalchemy import create_engine, inspect

# Substitua pelas credenciais e o nome do banco de dados
usuario = "seu_usuario"
senha = "sua_senha"
host = "localhost"
porta = "5432"
banco_conexao = "postgres"  # Banco de dados padrão para se conectar inicialmente

# URL de conexão ao PostgreSQL
DATABASE_URL = f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/{banco_conexao}"

# Crie a engine de conexão
engine = create_engine(DATABASE_URL)

# Listar os bancos de dados
with engine.connect() as connection:
    result = connection.execute("SELECT datname FROM pg_database WHERE datistemplate = false;")
    databases = [row['datname'] for row in result]
    print("Bancos de Dados:", databases)

# Se o banco de dados tpchdb existir, conecte-se a ele e liste as tabelas
if "tpchdb" in databases:
    DATABASE_URL = f"postgresql+psycopg2://{usuario}:{senha}@{host}:{porta}/tpchdb"
    engine_tpchdb = create_engine(DATABASE_URL)

    # Conecte-se ao banco de dados
    with engine_tpchdb.connect() as connection:
        print("Conectado ao banco de dados tpchdb!")

    # Inspecione e liste as tabelas
    inspector = inspect(engine_tpchdb)
    tables = inspector.get_table_names()
    print("Tabelas no banco de dados tpchdb:", tables)
else:
    print("O banco de dados 'tpchdb' não existe.")


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "tpch"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "tpch"

(Background on this error at: https://sqlalche.me/e/20/e3q8)